### Probabilistic Principal Component Analysis with Automatic Relevant Determination using Automatic Differentiation Variational Inference

In [1]:
from advi import *

**loading data :**

In [2]:
data = pd.read_csv('Data/interp_taxis_df.csv')

X = data.to_numpy()

data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y41,y42,y43,y44,y45,y46,y47,y48,y49,y50
0,-8.612145,-8.612134,-8.612223,-8.612532,-8.612801,-8.613046,-8.613043,-8.613072,-8.613258,-8.613715,...,41.157026,41.156311,41.156119,41.156069,41.156064,41.156064,41.156060,41.156049,41.156039,41.156028
1,-8.606979,-8.607039,-8.607198,-8.607731,-8.607784,-8.607565,-8.606649,-8.605264,-8.604333,-8.604351,...,41.128113,41.125798,41.124393,41.122211,41.120273,41.120069,41.119872,41.118261,41.116904,41.116626
2,-8.610966,-8.610454,-8.610069,-8.609403,-8.609149,-8.610050,-8.612146,-8.620671,-8.623431,-8.624752,...,41.150607,41.150597,41.150705,41.151157,41.152270,41.153323,41.154262,41.155045,41.155438,41.154813
3,-8.628435,-8.628425,-8.628395,-8.628308,-8.628220,-8.628133,-8.628084,-8.628048,-8.627616,-8.627147,...,41.162637,41.162638,41.162643,41.162658,41.162667,41.162657,41.162644,41.162629,41.162621,41.162616
4,-8.589447,-8.589453,-8.589460,-8.589468,-8.589480,-8.589473,-8.589416,-8.589294,-8.588715,-8.587755,...,41.148765,41.148765,41.148765,41.148771,41.148778,41.148784,41.148791,41.148797,41.148858,41.148999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-8.585739,-8.585772,-8.585788,-8.585791,-8.585785,-8.585893,-8.586218,-8.586846,-8.588049,-8.589561,...,41.147962,41.147880,41.147913,41.148425,41.148906,41.149186,41.149272,41.149654,41.150093,41.150430
49996,-8.566560,-8.567813,-8.576086,-8.581897,-8.583002,-8.579476,-8.581436,-8.592803,-8.608128,-8.621424,...,41.156434,41.163791,41.166883,41.169010,41.169176,41.169500,41.171482,41.172771,41.172898,41.172777
49997,-8.611488,-8.611502,-8.611520,-8.611315,-8.611305,-8.611181,-8.610945,-8.610891,-8.610909,-8.610899,...,41.143696,41.143869,41.144323,41.144471,41.144064,41.143372,41.142911,41.142721,41.142707,41.142708
49998,-8.589258,-8.589376,-8.589684,-8.590558,-8.591468,-8.592414,-8.593210,-8.593956,-8.594373,-8.594807,...,41.152742,41.151991,41.151240,41.150798,41.150341,41.149842,41.149580,41.149554,41.149579,41.149620


**model :**

In [3]:
latent_dim = 100

advi = ADVI(PPCA_ARD(X, latent_dim), mc_size=1)

advi.fit(X)

100%|██████████| 100/100 [00:19<00:00,  5.07it/s]
